# Imports

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Constants

In [2]:
torch.manual_seed(1337)
split_pct = 0.9
block_size = 256
batch_size = 64
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4 #1e-3
eval_iters = 200
n_embd = 64 #384
n_layer = 6 #4 # Number of block layers
n_head = 6 #4
dropout = 0.2

In [3]:
# Set GPU device
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'
print(device)

mps


# Download Dataset

In [4]:
#!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [5]:
with open('input.txt', 'r', encoding='utf-8') as input:
    text = input.read()

In [6]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



# Obtain all characters

In [7]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("Number of tokens: {}".format(vocab_size))

Number of tokens: 65


# Encoder/Decoder

In [8]:
encoder = { ch:i for i,ch in enumerate(chars) }
decoder = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [encoder[c] for c in s]
decode = lambda l: ''.join([decoder[i] for i in l])

In [9]:
print(encode("hello"))
print(decode(encode("hello")))

[46, 43, 50, 50, 53]
hello


# Tokenization

In [10]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data)

torch.Size([1115394]) torch.int64
tensor([18, 47, 56,  ..., 45,  8,  0])


# Split Data into Training & Validation

In [11]:
split = int(len(data)*split_pct)
train = data[:split]
valid = data[split:]

In [12]:
len(data), train.shape[0] + valid.shape[0], train.shape, valid.shape

(1115394, 1115394, torch.Size([1003854]), torch.Size([111540]))

# Data loader

In [13]:
def get_batch(split=None):
    # generate a small batch of data of inputs x and targets y
    data = train if split == 'train' else valid
    # Obtains blocks from random places in the data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    # y here is the target - the next token for any given point in x
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

# Model

In [14]:
# Obtains average loss over multiple batches
@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, T = get_batch(split)
            logits, loss = m(X, T)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

In [15]:
# This was my approach - was wrong. I think because n_embd * num_heads is meaningless? But got 1.97 loss - same as correct approach, so maybe it all works in the end as long as your dimensions are correct
# class MultiHeadAttention(nn.Module):

#     def __init__(self, num_heads, head_size):
#         super().__init__()
#         self.keys = nn.Linear(n_embd, n_embd * num_heads, bias=False) # (C, C * num_heads)
#         self.query = nn.Linear(n_embd, n_embd * num_heads, bias=False) # (C, C * num_heads)
#         self.value = nn.Linear(n_embd, n_embd * num_heads, bias=False) # (C, C * num_heads)
#         self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
#         self.dropout1 = nn.Dropout(dropout)
#         self.proj = nn.Linear(num_heads * n_embd, n_embd) # (C, C)
#         self.dropout2 = nn.Dropout(dropout)

#     def forward(self, x):
#         # x has shape (B,T,C) - because of initial embeddings
#         B, T, C = x.shape
#         k = self.keys(x) # (B,T,C) @ (C, C * num_heads)  --> (B,T,C * num_heads)
#         q = self.query(x) # (B,T,C) @ (C, C * num_heads)  --> (B,T,C * num_heads)
#         # affinities
#         wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B,T,kC) @ (B,kC,T) ---> (B,T,T)
#         wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
#         wei = F.softmax(wei, dim=-1) # (B, T, T)
#         wei = self.dropout1(wei)
#         v = self.value(x) # (B,T,C) @ (C, C * num_heads)  --> (B,T,C * num_heads)
#         out = wei @ v # (B, T, T) @ (B,T,kC) --> (B, T, kC)
#         out = self.dropout2(self.proj(out)) # (B, T, kC) @ (kC, C) --> (B, T, C)
#         return out

In [16]:
# This is copied from NanoGPT repo, fixing above mistakes. Loss here was 1.96
class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.c_attn = nn.Linear(n_embd, 3 * n_embd, bias=False) # (C, 3C)
        self.register_buffer('tril', torch.tril(torch.ones(1, 1, block_size, block_size)))
        self.dropout1 = nn.Dropout(dropout)
        self.proj = nn.Linear(n_embd, n_embd) # (C, C)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x):
        # x has shape (B,T,C) - because of initial embeddings
        B, T, C = x.shape

        q, k, v = self.c_attn(x).split(n_embd, dim=2) # (B, T, C) @ (C, 3C) --> (B, T, 3C) splits -> (B, T, C)
        k = k.view(B, T, self.num_heads, C // self.num_heads).transpose(1,2) # (B, nh, T, hs)
        q = q.view(B, T, self.num_heads, C // self.num_heads).transpose(1,2) # (B, nh, T, hs)
        v = v.view(B, T, self.num_heads, C // self.num_heads).transpose(1,2) # (B, nh, T, hs)

        # affinities
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, nh, T, hs) @ (B, nh, hs, T) ---> (B,nh,T,T)
        wei = wei.masked_fill(self.tril[:, :, :T, :T] == 0, float('-inf')) # (B, nh, T, T)
        wei = F.softmax(wei, dim=-1) # (B, nh, T, T)
        wei = self.dropout1(wei) # (B, nh, T, T)
        out = wei @ v # (B, nh, T, T) @ (B, nh, T, hs) --> (B, nh, T, hs)
        out = out.transpose(1, 2).contiguous().view(B,T,C) # (B, T, nh, hs) -> (B, T, C)
        out = self.dropout2(self.proj(out)) # (B, T, C) @ (C, C) --> (B, T, C)
        return out

In [17]:
class FeedForward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd), # Projection layer
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [18]:
class Block(nn.Module):

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        self.sa = MultiHeadAttention(n_head)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [19]:
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # Final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tkn_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tkn_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get predictions
            logits, loss = self(idx_cond)
            # focus only on last time step
            logits = logits[:, -1, :]
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # )B, C)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [20]:
m = BigramLanguageModel()
m.to(device)

BigramLanguageModel(
  (token_embedding_table): Embedding(65, 64)
  (position_embedding_table): Embedding(256, 64)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (c_attn): Linear(in_features=64, out_features=192, bias=False)
        (dropout1): Dropout(p=0.2, inplace=False)
        (proj): Linear(in_features=64, out_features=64, bias=True)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=64, out_features=256, bias=True)
          (1): ReLU()
          (2): Linear(in_features=256, out_features=64, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    )
    (1): Block(
      (sa): MultiHeadAttention(
        (c_attn): Linear(in_features=64, out_features=192, bias=False)
        (dropout1): Dropout(p=0.2, in

In [21]:
# NUmber of parameters in model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

0.224065 M parameters


# Training

In [22]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [23]:
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.3489, val loss 4.3546
step 500: train loss 2.3975, val loss 2.4139
step 1000: train loss 2.1514, val loss 2.1973
step 1500: train loss 1.9852, val loss 2.0697
step 2000: train loss 1.8762, val loss 1.9977
step 2500: train loss 1.8035, val loss 1.9329
step 3000: train loss 1.7508, val loss 1.8956
step 3500: train loss 1.7024, val loss 1.8494
step 4000: train loss 1.6631, val loss 1.8312
step 4500: train loss 1.6288, val loss 1.7962
step 4999: train loss 1.6032, val loss 1.7724


In [24]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))



HEver vagans! theirser pre; as but sordagle,
For thessessty three talk some,
No making tongs Hatche seectonedh your gettle of this
devand the man-somme longsion hids,
Or of the parce ove! If ave ister.
QUE MARGARET:
And I wam you she worth hath sucht causdsrstlssddndlslsssssglssslrdsdsssfsssnsgssssgsssddsrrsssgsssllsldsdlselssssgsscsgsllsgllsrgsssslstsssssgrnlsslsslsscsfsnltlnrs lslstdsslcbdsnsssssnnsgndssslssssssgssssmlrdsnsssdlppnsrrssdsssfnslplsllsslnsspssslcssssnlellsdsssls dsssslslssssdssd


# Save the model

In [27]:
import time
torch.save(m.state_dict(), './models/tiny_shakespeare_{}.pt'.format(time.strftime("%Y-%m-%d-%H:%M")))